# Tutorial: Basic Setup and Your First Call

This interactive tutorial will walk you through setting up Phony and making your first voice AI call. We'll cover:

1. Environment configuration
2. Service startup
3. Agent creation
4. Making test calls
5. Monitoring and debugging

## Prerequisites

Before starting, ensure you have:
- Docker and Docker Compose installed
- Twilio account with phone number
- OpenAI API key with Realtime API access
- Internet connection for API calls


## Step 1: Environment Setup

First, let's check if all required environment variables are configured:

In [ ]:
import os
from pathlib import Path

# Check for .env file
env_file = Path('.env')
if env_file.exists():
    print("✅ .env file found")
    
    # Load and check required variables
    with open(env_file) as f:
        env_content = f.read()
    
    required_vars = [
        'TWILIO_ACCOUNT_SID',
        'TWILIO_AUTH_TOKEN', 
        'TWILIO_PHONE_NUMBER',
        'OPENAI_API_KEY'
    ]
    
    for var in required_vars:
        if var in env_content and not env_content.split(f'{var}=')[1].split('\n')[0].strip() == '':
            print(f"✅ {var} configured")
        else:
            print(f"❌ {var} missing or empty")
else:
    print("❌ .env file not found. Please copy .env.example to .env and configure.")

If any variables are missing, create your `.env` file:

In [ ]:
# Create .env template if missing
env_template = '''
# Twilio Configuration
TWILIO_ACCOUNT_SID=your_account_sid_here
TWILIO_AUTH_TOKEN=your_auth_token_here
TWILIO_PHONE_NUMBER=+1234567890

# OpenAI Configuration
OPENAI_API_KEY=sk-your-key-here
OPENAI_MODEL=gpt-4o-realtime-preview
OPENAI_VOICE=alloy

# Application Settings
HOST=localhost
PORT=24187
SYSTEM_PROMPT=You are a helpful assistant
'''

if not Path('.env').exists():
    with open('.env', 'w') as f:
        f.write(env_template.strip())
    print("📝 Created .env template. Please edit with your credentials.")
else:
    print("📄 .env file already exists.")

## Step 2: Start Services

Now let's start the Phony services using Docker Compose:

In [ ]:
import subprocess
import time

def run_command(cmd, description):
    """Run a shell command and display results."""
    print(f"🔄 {description}...")
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        if result.returncode == 0:
            print(f"✅ {description} completed")
            if result.stdout.strip():
                print(f"Output: {result.stdout.strip()}")
        else:
            print(f"❌ {description} failed")
            print(f"Error: {result.stderr.strip()}")
        return result.returncode == 0
    except Exception as e:
        print(f"❌ Error running command: {e}")
        return False

# Start core services
success = run_command(
    "docker-compose up -d backend redis mongodb",
    "Starting Phony services"
)

if success:
    print("⏳ Waiting for services to initialize...")
    time.sleep(10)
    
    # Check service status
    run_command("docker-compose ps", "Checking service status")

## Step 3: Health Check

Let's verify that all services are running correctly:

In [ ]:
import requests
import json

def check_health():
    """Check the health of Phony services."""
    try:
        response = requests.get("http://localhost:24187/healthz", timeout=5)
        if response.status_code == 200:
            health_data = response.json()
            print("✅ Phony backend is healthy")
            print(f"   Uptime: {health_data.get('uptime', 0)} seconds")
            print(f"   Active calls: {health_data.get('activeCalls', 0)}")
            return True
        else:
            print(f"❌ Backend health check failed: {response.status_code}")
            return False
    except requests.exceptions.ConnectionError:
        print("❌ Cannot connect to backend. Is it running on port 24187?")
        return False
    except Exception as e:
        print(f"❌ Health check error: {e}")
        return False

backend_healthy = check_health()

if backend_healthy:
    print("\n🎉 All services are running! Ready to proceed.")
else:
    print("\n⚠️  Services may need more time to start. Try running the health check again.")

## Step 4: Configure Twilio Webhook

For Phony to receive calls, we need to configure the Twilio webhook:

In [ ]:
# Configure Twilio webhook automatically
webhook_success = run_command(
    "docker-compose run --rm demo python3 scripts/setup_twilio.py",
    "Configuring Twilio webhook"
)

if webhook_success:
    print("\n✅ Twilio webhook configured successfully!")
    print("Your phone number is now ready to receive AI calls.")
else:
    print("\n⚠️  Webhook configuration failed. You may need to set it up manually:")
    print("1. Go to Twilio Console > Phone Numbers")
    print("2. Select your phone number")
    print("3. Set Voice Webhook to: https://your-domain.com/receive_call")
    print("4. Set HTTP method to: POST")

## Step 5: Create Your First AI Agent

Let's create a simple customer service agent:

In [ ]:
# Agent configuration
agent_config = {
    "name": "Customer Service Bot",
    "system_prompt": "You are Sarah, a friendly customer service representative. Always introduce yourself and ask how you can help. Be professional but warm.",
    "voice": "alloy",
    "model": "gpt-4o-realtime-preview",
    "greeting_message": "Hello! I'm Sarah, your AI customer service assistant. How can I help you today?",
    "is_active": True
}

print("🤖 Agent Configuration:")
for key, value in agent_config.items():
    print(f"   {key}: {value}")

print("\n📝 This agent will:")
print("   • Introduce itself as Sarah")
print("   • Use a friendly, professional tone")
print("   • Ask how it can help")
print("   • Speak with the 'alloy' voice")

## Step 6: Test the System

Now let's run the full test suite to verify everything is working:

In [ ]:
# Run comprehensive test suite
print("🧪 Running Phony test suite...")
print("This may take a few minutes.\n")

tests_passed = run_command(
    "docker-compose run --rm demo python3 scripts/test_human_demo_suite.py",
    "Running main test suite"
)

if tests_passed:
    print("\n🎉 All tests passed! Your Phony installation is working correctly.")
else:
    print("\n⚠️  Some tests failed. Check the output above for details.")

## Step 7: Make Your First Call

Time to make your first AI call! We'll set up an interactive demo:

In [ ]:
print("📞 Ready to make your first call!")
print("\nYou have two options:")
print("\n1. AI calls you (outbound):")
print("   Run: docker-compose --profile human run --rm human-demo")
print("   Select option 1, consent to call, enter your number")

print("\n2. You call the AI (inbound):")
print("   Run: docker-compose --profile human run --rm human-demo") 
print("   Select option 2, choose personality, then call the number shown")

print("\n🎯 For this tutorial, let's try an outbound call:")

In [ ]:
# Interactive call setup
print("Setting up interactive demo...")
print("Note: This will start an interactive session. Follow the prompts.")

# This would typically launch the interactive demo
# For the notebook, we'll show the commands to run manually
demo_commands = [
    "# In a new terminal, run:",
    "docker-compose --profile human run --rm human-demo",
    "",
    "# Then:", 
    "# 1. Select 'AI calls human' (option 1)",
    "# 2. Type 'yes' to consent",
    "# 3. Enter your phone number (e.g., +15551234567)",
    "# 4. Select a scenario (1-4)",
    "# 5. Answer your phone when it rings!"
]

print("\n".join(demo_commands))

## Step 8: Monitor Your Call

While your call is active, you can monitor it through the dashboard:

In [ ]:
import webbrowser
from IPython.display import HTML, display

# Dashboard URL
dashboard_url = "http://localhost:24187/dashboard/"

print(f"📊 Dashboard URL: {dashboard_url}")
print("\nThe dashboard shows:")
print("   • Active calls in real-time")
print("   • Call transcriptions")
print("   • AI responses")
print("   • System metrics")
print("   • Supervisor controls")

# Create clickable link
display(HTML(f'<a href="{dashboard_url}" target="_blank">🔗 Open Dashboard</a>'))

# Try to open automatically
try:
    webbrowser.open(dashboard_url)
    print("\n🌐 Dashboard opened in your default browser")
except:
    print("\n⚠️  Could not auto-open browser. Please visit the URL manually.")

## Step 9: Advanced Configuration

Let's explore some advanced configuration options:

In [ ]:
# Available voices and their characteristics
voices = {
    "alloy": "Balanced, professional - good for business",
    "echo": "Clear, authoritative - great for tech support", 
    "fable": "Warm, storytelling - perfect for healthcare",
    "onyx": "Confident, persuasive - ideal for sales",
    "nova": "Friendly, conversational - customer service",
    "shimmer": "Energetic, upbeat - youth/entertainment"
}

print("🎙️ Available AI Voices:")
for voice, description in voices.items():
    print(f"   {voice}: {description}")

print("\n🎭 Pre-built Personalities:")
personalities = [
    "1. Professional Assistant - formal, business-focused",
    "2. Friendly Helper - warm, empathetic", 
    "3. Tech Expert - knowledgeable, detailed",
    "4. Sales Rep - persuasive, enthusiastic",
    "5. Emergency Responder - calm, authoritative"
]

for personality in personalities:
    print(f"   {personality}")

## Step 10: Troubleshooting

If you encounter issues, here are some debugging techniques:

In [ ]:
def troubleshoot():
    """Run troubleshooting checks."""
    print("🔍 Running troubleshooting checks...\n")
    
    # Check Docker
    print("1. Docker Status:")
    run_command("docker --version", "Docker version")
    run_command("docker-compose --version", "Docker Compose version")
    
    # Check services
    print("\n2. Service Status:")
    run_command("docker-compose ps", "Container status")
    
    # Check ports
    print("\n3. Port Availability:")
    import socket
    
    ports = [24187, 6380, 27017]
    for port in ports:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('localhost', port))
        if result == 0:
            print(f"   ✅ Port {port} is open")
        else:
            print(f"   ❌ Port {port} is not accessible")
        sock.close()
    
    # Check logs
    print("\n4. Recent Logs (last 10 lines):")
    run_command("docker-compose logs --tail=10 backend", "Backend logs")

# Run troubleshooting
troubleshoot()

## Congratulations! 🎉

You've successfully set up Phony and made your first voice AI call! 

### What you accomplished:
- ✅ Configured environment variables
- ✅ Started all required services
- ✅ Configured Twilio webhooks
- ✅ Ran comprehensive tests
- ✅ Made your first AI call
- ✅ Explored the dashboard

### Next Steps:
1. **Explore Features**: Try different AI personalities and voices
2. **Custom Prompts**: Create agents with specialized knowledge
3. **API Integration**: Build applications using Phony's REST API
4. **Multi-Agent**: Set up multiple agents for different scenarios
5. **Production**: Deploy to production with proper security

### Resources:
- 📚 [Complete Documentation](../guides/quickstart.md)
- 🔌 [API Reference](../api/index.md) 
- 🎓 [More Tutorials](./outbound-calls.ipynb)
- 💡 [Examples](../examples/customer-service.md)

Happy calling! 📞🤖